In [0]:
use catalog pricing_analytics;

insert into gold.reporting_fact_daily_pricing_gold
select dateDim.DATE_ID, marketDim.MARKET_ID, stateDim.STATE_ID, productDim.PRODUCT_ID, varietyDim.VARIETY_ID, silverfact.ROW_ID,
silverfact.ARRIVAL_IN_TONNES, silverfact.MAXIMUM_PRICE, silverfact.MINIMUM_PRICE, silverfact.MODAL_PRICE, 
current_timestamp(), current_timestamp()
from silver.daily_pricing_silver silverFact 
left outer join gold.reporting_dim_date_gold dateDim 
on silverFact.DATE_OF_PRICING = dateDim.CALENDAR_DATE
left outer join gold.reporting_dim_market_gold marketDim 
on silverFact.MARKET_NAME = marketDim.MARKET_NAME
left outer join gold.reporting_dim_state_gold stateDim 
on silverFact.STATE_NAME = stateDim.STATE_NAME
left outer join gold.reporting_dim_product_gold productDim 
on silverFact.PRODUCT_NAME = productDim.PRODUCT_NAME
and silverfact.PRODUCTGROUP_NAME = productDim.PRODUCTGROUP_NAME
left outer join gold.reporting_dim_variety_gold varietyDim 
on silverFact.VARIETY = varietyDim.VARIETY

where silverfact.lakehouse_updated_date >= 
(select nvl(max(PROCESSED_TABLE_DATE_TIME), "1900-01-01") from processrunlogs.deltalakehouse_process_runs
where PROCESS_STATUS= 'Completed' and PROCESS_NAME="reportingFactTableLoad")


In [0]:
insert into processrunlogs.deltalakehouse_process_runs(PROCESS_NAME, PROCESS_STATUS, PROCESSED_TABLE_DATE_TIME)
select  "reportingFactTableLoad", "Completed", max(lakehouse_updated_date) from silver.daily_pricing_silver